# Question 2- Predict Fradulent Transactions

___

Brainwaves Hackathon

Author: Ojaswi Kumar

Date: 09-12-2017
___

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from keras.utils import np_utils
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
%matplotlib inline

Using TensorFlow backend.


## Import Dataset

In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
frames = [data_train, data_test]
data = pd.concat(frames)
#data = data_train
#y = data['target']
X = data
#X = data.drop('return',axis=1)

## Visualize Data

In [3]:
X.head()

,cat_var_1,cat_var_10,cat_var_11,cat_var_12,cat_var_13,cat_var_14,cat_var_15,cat_var_16,cat_var_17,cat_var_18,...,cat_var_9,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,target,transaction_id
0,NaN,db,iq,ep,ip,db,ep,tn,tn,ep,...,tn,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,0.0,id_11
1,da,qt,iq,iq,hm,db,tn,tn,tn,tn,...,hr,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,0.0,id_33
2,gf,mm,ce,tn,hr,db,ep,tn,tn,ep,...,hr,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,0.0,id_51
3,NaN,es,iq,ep,ce,tn,ep,tn,tn,ep,...,ep,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,0.0,id_54
4,da,xy,hr,iq,hm,db,tn,tn,tn,tn,...,ep,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,0.0,id_62


In [4]:
X.describe()

,cat_var_19,cat_var_20,cat_var_21,cat_var_22,cat_var_23,cat_var_24,cat_var_25,cat_var_26,cat_var_27,cat_var_28,...,cat_var_41,cat_var_42,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,target
count,872444.000000,872444.000000,872444.000000,872444.000000,872444.000000,872444.000000,872444.000000,872444.000000,872444.000000,872444.000000,...,872444.000000,872444.0,8.724440e+05,872444.000000,872444.000000,8.724440e+05,8.724440e+05,8.724440e+05,8.724440e+05,348978.000000
mean,0.521605,0.478395,0.672051,0.321927,0.006022,0.999099,0.000110,0.000572,0.000005,0.000024,...,0.000001,0.0,1.868032e-05,0.160743,0.000017,4.428487e-05,7.441069e-06,1.523872e-05,2.128609e-05,0.107078
std,0.499533,0.499533,0.469466,0.467215,0.077369,0.030002,0.010489,0.023909,0.002141,0.004906,...,0.001071,0.0,1.764068e-03,0.131611,0.003382,2.439118e-03,6.072735e-04,1.521970e-03,1.959498e-03,0.309213
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,4.671053e-08,0.084805,0.000000,3.550000e-07,4.671053e-08,4.407895e-08,1.720602e-08,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,1.815789e-07,0.104261,0.000000,1.875000e-06,2.598684e-07,9.868421e-08,8.284379e-08,0.000000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,6.513158e-07,0.160833,0.000000,2.105000e-06,2.769737e-07,4.644737e-07,3.575028e-07,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.0,5.427632e-01,1.000000,1.000000,1.000000e+00,2.171053e-01,4.605263e-01,1.000000e+00,1.000000


## Check for missing values
### For Feature cleaning

In [5]:
print ('Column - Uniqe - Null')
for col in X:
    if X[col].isnull().any():
        print(col,len(X[col].unique()),X[col].isnull().any(),"--",X[col].isnull().sum())

Column - Uniqe - Null
cat_var_1 538 True -- 34474
cat_var_3 621 True -- 97215
cat_var_6 519 True -- 21943
cat_var_8 466 True -- 117876
target 3 True -- 523466


In [6]:
X = X.drop('transaction_id',axis=1)
X = X.drop('cat_var_1',axis=1)
X = X.drop('cat_var_3',axis=1)
X = X.drop('cat_var_6',axis=1)
X = X.drop('cat_var_8',axis=1)

In [7]:
print ('Column - Uniqe - Null')
for col in X:
    if not(X[col].isnull().any()):
        print (col,len(X[col].unique()),X[col].isnull().any())

Column - Uniqe - Null
cat_var_10 23 False
cat_var_11 5 False
cat_var_12 5 False
cat_var_13 52 False
cat_var_14 12 False
cat_var_15 2 False
cat_var_16 2 False
cat_var_17 2 False
cat_var_18 2 False
cat_var_19 2 False
cat_var_2 63 False
cat_var_20 2 False
cat_var_21 2 False
cat_var_22 2 False
cat_var_23 2 False
cat_var_24 2 False
cat_var_25 2 False
cat_var_26 2 False
cat_var_27 2 False
cat_var_28 2 False
cat_var_29 2 False
cat_var_30 2 False
cat_var_31 2 False
cat_var_32 2 False
cat_var_33 2 False
cat_var_34 2 False
cat_var_35 2 False
cat_var_36 2 False
cat_var_37 2 False
cat_var_38 1 False
cat_var_39 2 False
cat_var_4 2 False
cat_var_40 2 False
cat_var_41 2 False
cat_var_42 1 False
cat_var_5 2 False
cat_var_7 23 False
cat_var_9 5 False
num_var_1 17296 False
num_var_2 6050 False
num_var_3 14 False
num_var_4 1111 False
num_var_5 6206 False
num_var_6 15527 False
num_var_7 44177 False


In [8]:
keys = {}
le = LabelEncoder()
for col in X:
    #print col,len(X[col].unique()),X[col].isnull().any()
    #if False == X[col].isnull().any() and len(X[col].unique()) < 10 and 'cat_var' in col:
    if False == X[col].isnull().any() and 'cat_var' in col:
        X[col] = le.fit_transform(X[col])
        le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        keys[col] = le_name_mapping
        #print col," - OK"

In [ ]:
for x in keys:
    print(x)

In [9]:
X = data_train
X = X.drop('transaction_id',axis=1)
X = X.drop('cat_var_1',axis=1)
X = X.drop('cat_var_3',axis=1)
X = X.drop('cat_var_6',axis=1)
X = X.drop('cat_var_8',axis=1)

In [10]:
def convert2(x,col):
    return keys[col][x]

for col in X:
    #print(X[col])
    if col in keys.keys():
        X[col] = [convert2(a,col) for a in X[col]]

In [ ]:
X['target'].unique()

In [11]:
y = X['target']
X = X.drop('target',axis=1)

In [14]:
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),algorithm="SAMME",n_estimators=200)
#bdt = AdaBoostClassifier(RandomForestClassifier(max_depth=2, random_state=2),algorithm="SAMME.R",n_estimators=200) 
#Score: 0.711 ^^

In [15]:
bdt.fit(X,y)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=None)

In [ ]:
clf = GaussianNB()
clf.fit(X,y)

In [ ]:
clf = SVC()
clf.fit(X,y)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X, y) 

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=10, alpha=1, random_state=1) 
mlp.fit(X, y)

In [16]:
X_test = data_test.drop('transaction_id',axis=1)
X_test = X_test.drop('cat_var_1',axis=1)
X_test = X_test.drop('cat_var_3',axis=1)
X_test = X_test.drop('cat_var_6',axis=1)
X_test = X_test.drop('cat_var_8',axis=1)

In [17]:
def convert2(x,col):
    return keys[col][x]

for col in X_test:
    #print(X[col])
    if col in keys.keys():
        X_test[col] = [convert2(a,col) for a in X_test[col]]

In [18]:
#y_test = clf.predict_proba(X_test)
#y_test = neigh.predict_proba(X_test)
y_test = bdt.predict_proba(X_test)

In [19]:
y_test

array([[ 0.56366985,  0.43633015],
       [ 0.56153836,  0.43846164],
       [ 0.56188588,  0.43811412],
       ..., 
       [ 0.56195133,  0.43804867],
       [ 0.56188588,  0.43811412],
       [ 0.56196226,  0.43803774]])

In [20]:
y_test0 = pd.DataFrame(y_test[:,0])
y_test1 = pd.DataFrame(y_test[:,1])
data_test = pd.read_csv('test.csv')
df = data_test['transaction_id'].to_frame()

In [21]:
df.head()

,transaction_id
0,id_1
1,id_6
2,id_9
3,id_14
4,id_15


In [22]:
#result0 = pd.concat([df, y_test0], axis=1)
result1 = pd.concat([df, y_test1], axis=1)

In [23]:
#result0.columns = ['transaction_id','target']
#result0.to_csv('test_results0.csv',index=False)
result1.columns = ['transaction_id','target']
result1.to_csv('test_results1.csv',index=False)